In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


# Load the CSV file into a DataFrame
df = pd.read_csv('HW3_Macro_Data.csv')
df.set_index('Date', inplace=True)

# Display the first few rows
print(df.head())

              SP500       NDX       DJI    EWJ    EEM    EFA     GLD    USO  \
Date                                                                          
4/30/2025   5569.06  19571.02  40669.36  71.43  43.76  84.75  303.77  63.56   
3/31/2025   5611.85  19278.45  42001.76  68.56  43.70  81.73  288.14  77.33   
2/28/2025   5954.50  20884.41  43840.91  68.47  43.21  81.58  263.27  75.22   
1/31/2025   6040.53  21478.05  44544.66  68.31  42.72  79.24  258.56  78.02   
12/31/2024  5881.63  21012.17  42544.22  67.10  41.82  75.61  242.13  75.55   

              IEF    HYG    VNQ     SPTSX    SXXP      ASX    UUP     FXE  \
Date                                                                        
4/30/2025   96.07  78.60  88.33  24841.68  527.48  4594.05  27.41  104.63   
3/31/2025   95.37  78.89  90.54  24917.50  533.92  4623.62  28.54   99.90   
2/28/2025   95.32  80.13  93.90  25393.45  557.19  4754.32  29.39   95.84   
1/31/2025   93.02  79.72  90.55  25533.10  539.53  4710.58  2

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. Select features and scale them
scaler = StandardScaler()
X_raw = df[['CPI', 'FDFD']]
X_scaled = scaler.fit_transform(X_raw)

results = []

# 2. Loop over each column as the dependent variable
for col in df.columns:
    if col in ['CPI', 'FDFD']:
        continue  # skip the regressors themselves
    y = df[col]
    
    # 3. Fit the model using scaled X
    model = LinearRegression()
    model.fit(X_scaled, y)

    # 4. Extract betas and R^2
    beta_cpi, beta_fdfd = model.coef_
    r2 = model.score(X_scaled, y)
    
    results.append({
        'variable': col,
        'beta_CPI':     beta_cpi,
        'beta_FDFD':    beta_fdfd,
        'r_squared':    r2
    })

# 5. Build and print the summary DataFrame
results_df = pd.DataFrame(results).set_index('variable')
print(results_df)


             beta_CPI    beta_FDFD  r_squared
variable                                     
SP500      400.043577   860.402565   0.621487
NDX       1640.593987  3488.516466   0.600921
DJI       2914.459963  6084.893378   0.629299
EWJ          1.756908     5.356460   0.378597
EEM          1.333752    -0.895900   0.084578
EFA          1.690908     4.268699   0.340560
GLD          8.139553    19.646953   0.401763
USO        -12.886155   -41.550591   0.219282
IEF         -0.114199    -3.989423   0.273360
HYG         -1.569457    -3.897426   0.640494
VNQ          5.111503     3.094471   0.229099
SPTSX     1258.359826  2197.630428   0.647553
SXXP        17.026718    49.899069   0.535164
ASX         95.103321   315.602444   0.489766
UUP          0.369997     1.729826   0.614771
FXE         -2.255335    -7.693924   0.395432
